In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.tools.eval_measures import rmse
from sklearn.preprocessing import MinMaxScaler
from keras import Input
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.preprocessing.sequence import TimeseriesGenerator
import warnings
import urllib

warnings.filterwarnings("ignore")

# 데이터 다운로드
url = 'https://storage.googleapis.com/javalove93-samples/kaggle/corona2/covid_19_data.csv'
# data = np.genfromtxt(urllib.request.urlopen(url), delimiter=',', skip_header=1)
# df = pd.DataFrame(data)
!wget $url -O covid_19_data.csv
!wc -l covid_19_data.csv
!head covid_19_data.csv

Using TensorFlow backend.
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type

In [2]:
df = pd.read_csv("covid_19_data.csv")
df.ObservationDate = pd.to_datetime(df.ObservationDate)
df = df.set_index('ObservationDate')

# 데이터를 출력해 보면 Month와 #Passengers라는 두 개의 필드를 가진 데이터
print(df)
df.head()

FileNotFoundError: [Errno 2] File covid_19_data.csv does not exist: 'covid_19_data.csv'

In [ ]:
%pylab inline

In [ ]:
pylab.rcParams['figure.figsize'] = (20, 5)

def country_data(df, country):
    if country == "World":
        df_country = df.groupby(['ObservationDate']).sum()
    elif type(country) == type([]):
        df_country = df[df['Country/Region'].isin(country)].groupby(['ObservationDate']).sum()
    else:
        df_country = df[df['Country/Region'] == country].groupby(['Country/Region', 'ObservationDate']).sum()
    df_country = df_country.reset_index()
    df_country = df_country.set_index('ObservationDate')
    df_country['Pct_Change'] = df_country.Confirmed.pct_change()
    df_country['Daily_Confirmed'] = df_country.Confirmed.diff()
#     max_patients = df_country['Confirmed'].max()
    df_country['Patients'] = df_country['Confirmed'] - df_country['Recovered']
    return df_country

def draw_country_data(df_country, country=None):
    df_country.Confirmed.plot(grid=True)
    df_country.Deaths.plot(grid=True)
    df_country.Recovered.plot(grid=True)
    df_country.Patients.plot(grid=True)
    plt.legend()
    plt.xlabel("Date")
    if country:
        plt.title(country)
    else:
        plt.title(df_country["Country/Region"].unique())
    plt.show()

    df_country.Daily_Confirmed.plot(kind='bar')
    plt.legend()
    plt.xlabel("Date")
    plt.ylabel("New Confirmed")
    if country:
        plt.title(country)
    else:
        plt.title(df_country["Country/Region"].unique())
    plt.show()

    df_country.Pct_Change.plot(grid=True)
    plt.legend()
    plt.xlabel("Date")
    plt.ylabel("Ratio")
    if country:
        plt.title(country)
    else:
        plt.title(df_country["Country/Region"].unique())
    plt.show()



In [ ]:
dfc = country_data(df, "Israel")
draw_country_data(dfc)

In [ ]:
dfc = country_data(df, "South Korea")
draw_country_data(dfc)

In [ ]:
dfc = country_data(df, "Italy")
draw_country_data(dfc)

In [ ]:
# United States
d = country_data(df, "US")
draw_country_data(d)


In [ ]:
# Russia
d = country_data(df, "Russia")
draw_country_data(d)


In [ ]:
# World
d = country_data(df, "World")
draw_country_data(d, "World")

In [ ]:
def show_trend(df, countries, patients="Confirmed", new_patients="Daily_Confirmed", date="ObservationDate", is_sum=False):
    if type(countries) == type(""):
        countries = [countries]
    
    for country in countries:
        ndf = country_data(df, country).copy()
        ndf['aweek_sum'] = ndf[patients].rolling(window=14, axis=0).mean()
        if is_sum:
            ndf['aweek_new'] = ndf[new_patients].rolling(window=7, axis=0).sum()
        else:
            ndf['aweek_new'] = ndf[new_patients].rolling(window=7, axis=0).mean()
        ndf = ndf.dropna(axis=0)

        ndf = ndf.reset_index()
        ndf = ndf.set_index("aweek_sum")
        ndf.aweek_new.plot(grid=True, label=str(country))
        
    plt.xscale("log")
    plt.yscale("log")
    plt.legend()
    plt.title(str(countries))
    plt.xlabel("Total Patients")
    plt.ylabel("New Confirmed")
    plt.show()

pylab.rcParams['figure.figsize'] = (30, 16)
show_trend(df, ["Israel","South Korea", "Italy", "US", "Russia"])